# Kernel keeps restarting when I want to run all of the charts in `_report_route_dir_visuals`

In [1]:
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import google.auth
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

readable_dict = OmegaConf.load("readable2.yml")
credentials, project = google.auth.default()

import _report_route_dir_visuals

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# portfolio_name = "City and County of San Francisco"
portfolio_name =  "Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"

In [4]:
ROUTE_DIR_MONTH_FILE = GTFS_DATA_DICT.digest_tables.monthly_route_schedule_vp_report
ROUTE_DIR_QTR_FILE = "digest/quarterly_schedule_vp_metrics_report"

month_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_MONTH_FILE}.parquet",
    filters=[[("Portfolio Organization Name", "==", portfolio_name)]],
)

qtr_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_QTR_FILE}.parquet",
    filters=[[("Portfolio Organization Name", "==", portfolio_name)]],
)

## The charts won't display -> The cell keeps buffering with the * sign.

In [5]:
routes_list = month_df["Route"].unique().tolist()

route_dropdown = alt.binding_select(
        options=routes_list,
        name="Routes: ",
    )
    # Column that controls the bar charts
xcol_param = alt.selection_point(
        fields=["Route"], value=routes_list[0], bind=route_dropdown
    )

In [6]:
len(month_df)

285587

In [7]:
month_df.Date.unique()

array(['2023-04-12T00:00:00.000000000', '2023-06-14T00:00:00.000000000',
       '2023-07-12T00:00:00.000000000', '2023-08-15T00:00:00.000000000',
       '2023-09-13T00:00:00.000000000', '2023-10-11T00:00:00.000000000',
       '2023-11-15T00:00:00.000000000', '2023-12-13T00:00:00.000000000',
       '2024-01-17T00:00:00.000000000', '2024-02-14T00:00:00.000000000',
       '2024-03-13T00:00:00.000000000', '2024-04-17T00:00:00.000000000',
       '2024-05-22T00:00:00.000000000', '2024-06-12T00:00:00.000000000',
       '2024-07-17T00:00:00.000000000', '2024-08-14T00:00:00.000000000',
       '2024-09-18T00:00:00.000000000', '2024-10-16T00:00:00.000000000',
       '2024-11-13T00:00:00.000000000', '2024-12-11T00:00:00.000000000',
       '2025-01-15T00:00:00.000000000', '2025-02-12T00:00:00.000000000',
       '2025-03-12T00:00:00.000000000', '2025-04-16T00:00:00.000000000',
       '2025-05-14T00:00:00.000000000', '2023-03-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [8]:
len(qtr_df)

2425

In [9]:
# Charts
spatial_accuracy = (
        _report_route_dir_visuals.spatial_accuracy_chart(qtr_df[qtr_df.Period == "All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

avg_scheduled_min = (
       _report_route_dir_visuals.avg_scheduled_min_chart(qtr_df[qtr_df.Period == "All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [10]:
vp_per_minute = (
        _report_route_dir_visuals.vp_per_minute_chart(qtr_df[qtr_df.Period == "All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
speed = (
        _report_route_dir_visuals.speed_chart(month_df)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [11]:
sched_vp_per_min = (
        _report_route_dir_visuals.sched_vp_per_min_chart(month_df[month_df.Period == "All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

text_dir0 = (
        _report_route_dir_visuals.text_chart(month_df, 0)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [12]:
text_dir1 = (
        _report_route_dir_visuals.text_chart(month_df, 1)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

timeliness_dir0 =(
        _report_route_dir_visuals.timeliness_chart(month_df[month_df.Period == "All Day"], 0)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [13]:
timeliness_dir1 =(
        _report_route_dir_visuals.timeliness_chart(month_df[month_df.Period == "All Day"], 1)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

n_scheduled_dir0 = (
        _report_route_dir_visuals.total_scheduled_trips_chart(month_df[month_df.Period == "All Day"], 0)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [14]:
n_scheduled_dir1 = (
        _report_route_dir_visuals.total_scheduled_trips_chart(month_df[month_df.Period == "All Day"], 1)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [15]:
n_freq_dir0 = (
        _report_route_dir_visuals.headway_chart(month_df[month_df.Period != "All Day"], 0)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

n_freq_dir1 = (
         _report_route_dir_visuals.headway_chart(month_df[month_df.Period != "All Day"], 1)
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

In [16]:
data_quality = _report_route_dir_visuals.divider_chart(
        month_df, readable_dict.data_quality_graph.title
    )
rider_quality = _report_route_dir_visuals.divider_chart(
        month_df, readable_dict.rider_quality_graph.title
    )
summary = _report_route_dir_visuals.divider_chart(month_df, readable_dict.summary_graph.title)

In [17]:
chart_summary_list = [
        summary,
        text_dir0,
        text_dir1]

In [18]:
summary_chart = alt.vconcat(*chart_summary_list)

In [19]:
summary_chart

alt.VConcatChart(...)

In [20]:
data_quality_list = [data_quality,
        spatial_accuracy,
        vp_per_minute,
        sched_vp_per_min,
    ]

In [21]:
data_quality_chart = alt.vconcat(*data_quality_list)

In [ ]:
data_quality_chart

In [ ]:
rider_quality_list1 =[rider_quality,
        avg_scheduled_min,
        timeliness_dir0,
        timeliness_dir1,]

In [ ]:
rider_quality_chart1 = alt.vconcat(*rider_quality_list1)

In [ ]:
rider_quality_chart1

In [ ]:
        n_freq_dir0,
        n_scheduled_dir0,
        n_freq_dir1,
        n_scheduled_dir1,
        speed] 

In [ ]:
rider_quality_chart

In [ ]:
# chart = (_report_route_dir_visuals.route_filter(route_dir_qtr_df,route_dir_month_df))

In [ ]:
chart

## Text charts are causing the kernel to die for some notebooks.

In [ ]:
#_report_route_dir_visuals.route_filter(route_dir_qtr_df, route_dir_month_df)

In [ ]:
route_dir_month_df.shape

In [ ]:
route_dir_month_df = route_dir_month_df.loc[route_dir_month_df["Direction (0/1)"] == 0]

In [ ]:
len(route_dir_month_df)

In [ ]:
route_dir_month_df.head(1)

In [ ]:
route_stats_df = _report_route_dir_visuals.reshape_route_stats(route_dir_month_df)

In [ ]:
len(route_stats_df)

In [ ]:
len(route_stats_df.drop_duplicates())

In [ ]:
route_stats_df = route_stats_df.drop_duplicates()

In [ ]:
# Reshape dataframe before plotting
melt1 = route_stats_df.melt(
        id_vars=[
            "Date",
            "Route",
            "Direction",
            "Direction (0/1)",
        ],
        value_vars=[
            "Average Stop Distance (Miles)",
            "# Scheduled Trips",
            "Gtfs Availability",
            "Peak Avg Speed (Mph)",
            "Peak Scheduled Trips",
            "Peak Headway (Minutes)",
            "Offpeak Avg Speed (Mph)",
            "Offpeak Scheduled Trips",
            "Offpeak Headway (Minutes)",
        ],
    )

In [ ]:
route_stats_df["Date"].unique()

In [ ]:
len(melt1)

In [ ]:
text_dir0 = (
        _report_route_dir_visuals.text_chart(route_dir_month_df, 0)
    )

In [ ]:
text_dir0